In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer

In [2]:
def summarize_text_bart(document):
    # Load pre-trained BART model and tokenizer
    model_name = "facebook/bart-large-cnn"
    model = BartForConditionalGeneration.from_pretrained(model_name)
    tokenizer = BartTokenizer.from_pretrained(model_name)

    # Tokenize and encode the document
    inputs = tokenizer.encode("summarize: " + document, return_tensors="pt", max_length=1024, truncation=True)

    # Generate summary
    summary_ids = model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

if __name__ == "__main__":
    # Example document
    input_document = """
    Adani is also among the most controversial of India’s billionaires for his association with the Bharatiya Janata Party (BJP). His close relationship with the party is not coincidental: Adani frequently refers to his business strategy as motivated by “nation building,” which the Adani Group describes on its website as “helping build world-class infrastructure capabilities to help accelerate the growth of India.” Mundra Port and its associated Adani Special Economic Zone, the central components of Adani’s business empire, were attained and developed in cooperation with the Gujarat state government. The BJP led the Gujarat state government during key moments of the Adani Group’s growth, and the relationship resulted in the symbiotic rise of both the BJP and the Adani Group.
    """

    # Summarize the document
    output_summary = summarize_text_bart(input_document)

    # Print the results
    print("Original Document:")
    print(input_document)
    print("\nSummary:")
    print(output_summary)

Original Document:

    Adani is also among the most controversial of India’s billionaires for his association with the Bharatiya Janata Party (BJP). His close relationship with the party is not coincidental: Adani frequently refers to his business strategy as motivated by “nation building,” which the Adani Group describes on its website as “helping build world-class infrastructure capabilities to help accelerate the growth of India.” Mundra Port and its associated Adani Special Economic Zone, the central components of Adani’s business empire, were attained and developed in cooperation with the Gujarat state government. The BJP led the Gujarat state government during key moments of the Adani Group’s growth, and the relationship resulted in the symbiotic rise of both the BJP and the Adani Group.
    

Summary:
Adani is also among the most controversial of India’s billionaires for his association with the Bharatiya Janata Party (BJP) His close relationship with the party is not coinciden

In [3]:
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")

# by default encoder-attention is `block_sparse` with num_random_blocks=3, block_size=64
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv")

# decoder attention type can't be changed & will be "original_full"
# you can change `attention_type` (encoder only) to full attention like this:
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", attention_type="original_full")

# you can change `block_size` & `num_random_blocks` like this:
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", block_size=16, num_random_blocks=2)

text = """
    Adani is also among the most controversial of India’s billionaires for his association with the Bharatiya Janata Party (BJP). His close relationship with the party is not coincidental: Adani frequently refers to his business strategy as motivated by “nation building,” which the Adani Group describes on its website as “helping build world-class infrastructure capabilities to help accelerate the growth of India.” Mundra Port and its associated Adani Special Economic Zone, the central components of Adani’s business empire, were attained and developed in cooperation with the Gujarat state government. The BJP led the Gujarat state government during key moments of the Adani Group’s growth, and the relationship resulted in the symbiotic rise of both the BJP and the Adani Group.
    """
inputs = tokenizer(text, return_tensors='pt')
prediction = model.generate(**inputs)
prediction = tokenizer.batch_decode(prediction)

# Print the results

print("Original Document:")
print(text)
print("\nSummary:")
print(prediction)

Original Document:

    Adani is also among the most controversial of India’s billionaires for his association with the Bharatiya Janata Party (BJP). His close relationship with the party is not coincidental: Adani frequently refers to his business strategy as motivated by “nation building,” which the Adani Group describes on its website as “helping build world-class infrastructure capabilities to help accelerate the growth of India.” Mundra Port and its associated Adani Special Economic Zone, the central components of Adani’s business empire, were attained and developed in cooperation with the Gujarat state government. The BJP led the Gujarat state government during key moments of the Adani Group’s growth, and the relationship resulted in the symbiotic rise of both the BJP and the Adani Group.
    

Summary:
["<s> in this paper, we address the question of whether or notdani s proposal to build a <unk> tunnel '' between the earth and the atmosphere is consistent with the laws of thermo

# BART

In [5]:
from transformers import pipeline, BartTokenizer, BartForConditionalGeneration

text = """
    Adani is also among the most controversial of India’s billionaires for his association with the Bharatiya Janata Party (BJP). His close relationship with the party is not coincidental: Adani frequently refers to his business strategy as motivated by “nation building,” which the Adani Group describes on its website as “helping build world-class infrastructure capabilities to help accelerate the growth of India.” Mundra Port and its associated Adani Special Economic Zone, the central components of Adani’s business empire, were attained and developed in cooperation with the Gujarat state government. The BJP led the Gujarat state government during key moments of the Adani Group’s growth, and the relationship resulted in the symbiotic rise of both the BJP and the Adani Group.
    """
# Load BART model and tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Tokenize input text
inputs = tokenizer(text, return_tensors='pt')

# Generate summary
summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)



print("Original Document:")
print(text)
print("BART Summary:", summary)




from rouge_score import rouge_scorer

# Original text and generated summary
reference_text = text
generated_summary = summary

# Calculate ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_text, generated_summary)


print('\n\n\n')
# Print ROUGE scores
print("ROUGE-1 Precision:", scores['rouge1'].precision)
print("ROUGE-1 Recall:", scores['rouge1'].recall)
print("ROUGE-1 F1 Score:", scores['rouge1'].fmeasure)

print("\nROUGE-2 Precision:", scores['rouge2'].precision)
print("ROUGE-2 Recall:", scores['rouge2'].recall)
print("ROUGE-2 F1 Score:", scores['rouge2'].fmeasure)

print("\nROUGE-L Precision:", scores['rougeL'].precision)
print("ROUGE-L Recall:", scores['rougeL'].recall)
print("ROUGE-L F1 Score:", scores['rougeL'].fmeasure)


C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



Original Document:

    Adani is also among the most controversial of India’s billionaires for his association with the Bharatiya Janata Party (BJP). His close relationship with the party is not coincidental: Adani frequently refers to his business strategy as motivated by “nation building,” which the Adani Group describes on its website as “helping build world-class infrastructure capabilities to help accelerate the growth of India.” Mundra Port and its associated Adani Special Economic Zone, the central components of Adani’s business empire, were attained and developed in cooperation with the Gujarat state government. The BJP led the Gujarat state government during key moments of the Adani Group’s growth, and the relationship resulted in the symbiotic rise of both the BJP and the Adani Group.
    
BART Summary: Adani is among the most controversial of India’s billionaires for his association with the Bharatiya Janata Party. His close relationship with the party is not coincidental. 

# T5

In [6]:
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration

text = """
    Adani is also among the most controversial of India’s billionaires for his association with the Bharatiya Janata Party (BJP). His close relationship with the party is not coincidental: Adani frequently refers to his business strategy as motivated by “nation building,” which the Adani Group describes on its website as “helping build world-class infrastructure capabilities to help accelerate the growth of India.” Mundra Port and its associated Adani Special Economic Zone, the central components of Adani’s business empire, were attained and developed in cooperation with the Gujarat state government. The BJP led the Gujarat state government during key moments of the Adani Group’s growth, and the relationship resulted in the symbiotic rise of both the BJP and the Adani Group.
    """
# Load T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Tokenize input text
inputs = tokenizer("summarize: " + text, return_tensors='pt')

# Generate summary
summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Original Document:")
print(text)
print("T5 Summary:", summary)




from rouge_score import rouge_scorer

# Original text and generated summary
reference_text = text
generated_summary = summary

# Calculate ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_text, generated_summary)


print('\n\n\n')
# Print ROUGE scores
print("ROUGE-1 Precision:", scores['rouge1'].precision)
print("ROUGE-1 Recall:", scores['rouge1'].recall)
print("ROUGE-1 F1 Score:", scores['rouge1'].fmeasure)

print("\nROUGE-2 Precision:", scores['rouge2'].precision)
print("ROUGE-2 Recall:", scores['rouge2'].recall)
print("ROUGE-2 F1 Score:", scores['rouge2'].fmeasure)

print("\nROUGE-L Precision:", scores['rougeL'].precision)
print("ROUGE-L Recall:", scores['rougeL'].recall)
print("ROUGE-L F1 Score:", scores['rougeL'].fmeasure)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Original Document:

    Adani is also among the most controversial of India’s billionaires for his association with the Bharatiya Janata Party (BJP). His close relationship with the party is not coincidental: Adani frequently refers to his business strategy as motivated by “nation building,” which the Adani Group describes on its website as “helping build world-class infrastructure capabilities to help accelerate the growth of India.” Mundra Port and its associated Adani Special Economic Zone, the central components of Adani’s business empire, were attained and developed in cooperation with the Gujarat state government. The BJP led the Gujarat state government during key moments of the Adani Group’s growth, and the relationship resulted in the symbiotic rise of both the BJP and the Adani Group.
    
T5 Summary: Adani is among the most controversial billionaires for his association with the Bharatiya Janata Party (BJP) he frequently refers to his business strategy as motivated by 'natio

# Pegasus

In [7]:
from transformers import pipeline, PegasusTokenizer, PegasusForConditionalGeneration

text = """
    Adani is also among the most controversial of India’s billionaires for his association with the Bharatiya Janata Party (BJP). His close relationship with the party is not coincidental: Adani frequently refers to his business strategy as motivated by “nation building,” which the Adani Group describes on its website as “helping build world-class infrastructure capabilities to help accelerate the growth of India.” Mundra Port and its associated Adani Special Economic Zone, the central components of Adani’s business empire, were attained and developed in cooperation with the Gujarat state government. The BJP led the Gujarat state government during key moments of the Adani Group’s growth, and the relationship resulted in the symbiotic rise of both the BJP and the Adani Group.
    """
# Load Pegasus model and tokenizer
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-cnn_dailymail')
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-cnn_dailymail')

# Tokenize input text
inputs = tokenizer(text, return_tensors='pt')

# Generate summary
summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Original Document:")
print(text)
print("Pegasus Summary:", summary)



from rouge_score import rouge_scorer

# Original text and generated summary
reference_text = text
generated_summary = summary

# Calculate ROUGE scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_text, generated_summary)


print('\n\n\n')
# Print ROUGE scores
print("ROUGE-1 Precision:", scores['rouge1'].precision)
print("ROUGE-1 Recall:", scores['rouge1'].recall)
print("ROUGE-1 F1 Score:", scores['rouge1'].fmeasure)

print("\nROUGE-2 Precision:", scores['rouge2'].precision)
print("ROUGE-2 Recall:", scores['rouge2'].recall)
print("ROUGE-2 F1 Score:", scores['rouge2'].fmeasure)

print("\nROUGE-L Precision:", scores['rougeL'].precision)
print("ROUGE-L Recall:", scores['rougeL'].recall)
print("ROUGE-L F1 Score:", scores['rougeL'].fmeasure)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Original Document:

    Adani is also among the most controversial of India’s billionaires for his association with the Bharatiya Janata Party (BJP). His close relationship with the party is not coincidental: Adani frequently refers to his business strategy as motivated by “nation building,” which the Adani Group describes on its website as “helping build world-class infrastructure capabilities to help accelerate the growth of India.” Mundra Port and its associated Adani Special Economic Zone, the central components of Adani’s business empire, were attained and developed in cooperation with the Gujarat state government. The BJP led the Gujarat state government during key moments of the Adani Group’s growth, and the relationship resulted in the symbiotic rise of both the BJP and the Adani Group.
    
Pegasus Summary: Adani is among the most controversial of India’s billionaires for his association with the Bharatiya Janata Party ( BJP)<n>The BJP led the Gujarat state government during k